In [22]:
import os
from huggingface_hub import login
from datasets import load_dataset
import pandas as pd
from mlx_lm import load, generate, lora

In [2]:
login()

# Dataset preparation

In [3]:
dataset = load_dataset("jbeiroa/resume-summarization-dataset")
train = dataset["train"]
val = dataset["validation"]
test = dataset["test"]

README.md:   0%|          | 0.00/795 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/52.7M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/6.55M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/6.43M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11612 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1451 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1452 [00:00<?, ? examples/s]

In [7]:
PROMPT_TEMPLATE = (
    "Summarize the following resume in 3-4 sentences, focusing on key skills, experience, and education.\n\n"
)

In [23]:
def convert_split(split):
    return [
        {
            "prompt": PROMPT_TEMPLATE + example["resume"],
            "completion": example["summary"]
        }
        for example in split
        if example["resume"].strip() and example["summary"].strip()
    ]

In [24]:
train_mlx = convert_split(train)
val_mlx = convert_split(val)
test_mlx = convert_split(test)

In [26]:
import json

def save_jsonl(data, filename):
    with open(filename, "w") as f:
        for item in data:
            json.dump(item, f)
            f.write("\n")

save_jsonl(train_mlx, "/Users/juanbeiroa/Code/thereisnohr/data/finetuning/train.jsonl")
save_jsonl(val_mlx, "/Users/juanbeiroa/Code/thereisnohr/data/finetuning/valid.jsonl")
save_jsonl(test_mlx, "/Users/juanbeiroa/Code/thereisnohr/data/finetuning/test.jsonl")

# Finetuning

In [21]:
models_to_finetune = {
    "phi2": "microsoft/phi-2",
    "tinyllama": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "gemma2b": "mlx-community/gemma-2-2b-it",
    "llama3.2": "mlx-community/Llama-3.2-1B-Instruct-MLXTuned"
}

In [ ]:
!mlx_lm.lora \
    --model TinyLlama/TinyLlama-1.1B-Chat-v1.0 \
    --train \
    --data ~/Code/thereisnohr/data/finetuning \
    --adapter-path ~/Code/thereisnohr/adapters/tinyllama \
    --batch-size 2 \
    --num-layers 4 \
    --test